<a href="https://colab.research.google.com/github/DGkurnia/cnkproject/blob/main/BMLP_Derfansyah_Guswiranata_Kurnia_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
#Perpustakaan untuk pengelolaan data
import numpy as np
import pandas as pd
#Grafik
import matplotlib.pyplot as plt #grafik dasar
import seaborn as sns
import datetime

In [37]:
#pelabelan untuk klasifikasi
from sklearn.preprocessing import LabelBinarizer as lbina
from sklearn.preprocessing import LabelEncoder as lenco

In [5]:
#Bagian knn(p1)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix as cmatr, classification_report
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
#jika pakai multilabel
from sklearn.metrics import multilabel_confusion_matrix as mcmat

In [6]:
#Bagian klasifikasi 'randomforest'
from sklearn.ensemble import RandomForestClassifier

In [7]:
#Pengunduhan dataset ke google colab
from google.colab import files
#ke proses 'upload'

In [8]:
#proses 'upload'
uploaded = files.upload()

Saving hslclstr.csv to hslclstr.csv


In [9]:
#pengelolaan file
import os #untuk os
import time #waktu
import datetime #tanggal
import gc #pengunduhan file per batch

In [10]:
#Penyimpanan hasil sementara di google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
#inisiasi berat sampel
from sklearn.utils.class_weight import compute_sample_weight as csw #inspeksi berat dl

In [99]:
#Bagian multiprocessing melalui instalasi pool
import warnings #peringatan
import multiprocessing as mp #untuk multiprocessing
from multiprocessing import Pool
import joblib #Bagian Joblib

In [13]:
#penyimpanan hasil melalui dill
! pip install dill
import dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.3 MB/s eta 0:00:00


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [14]:
#Cek data
df = pd.read_csv('/content/hslclstr.csv')
dfc = df.copy() #jangan pakai data asli

In [15]:
#cek hasil
smpl = dfc.head(7)
inf = dfc.info() #informasi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   img                          64016 non-null  object 
 1   title                        64016 non-null  object 
 2   console                      64016 non-null  object 
 3   genre                        64016 non-null  object 
 4   publisher                    64016 non-null  object 
 5   total_sales                  64016 non-null  float64
 6   na_sales                     64016 non-null  float64
 7   jp_sales                     64016 non-null  float64
 8   pal_sales                    64016 non-null  float64
 9   other_sales                  64016 non-null  float64
 10  critic_score                 64016 non-null  float64
 11  release_date                 64016 non-null  object 
 12  last_update                  64016 non-null  object 
 13  kemungkinan gene

In [16]:
#inspeksi label untuk sampel
lbl = dfc['Label'].copy()
print(lbl.head(7))

0    game kurang berkenan dengan fokus pencarian ke...
1    game kurang berkenan dengan fokus pencarian ke...
2    game kurang berkenan dengan fokus pencarian ke...
3                game dengan tujuan utama pencari duit
4    game kurang berkenan dengan fokus pencarian ke...
5                game dengan tujuan utama pencari duit
6                game dengan tujuan utama pencari duit
Name: Label, dtype: object


Kolom ini menjadi basis sumbu y

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [17]:
#Inisiasi pembagian data
inspk = dfc.iloc[:,5:11].copy() #numerikal untuk klasifikasi kluster
inspk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   total_sales   64016 non-null  float64
 1   na_sales      64016 non-null  float64
 2   jp_sales      64016 non-null  float64
 3   pal_sales     64016 non-null  float64
 4   other_sales   64016 non-null  float64
 5   critic_score  64016 non-null  float64
dtypes: float64(6)
memory usage: 2.9 MB


In [18]:
#Sumbu x
xcls = inspk.copy() #pakai numerik

Sumbu x  **(xcls)** adalah nilai numerikal dari tabel yang terdiri dari
* Penjualan dari lima indikator (total, spesifika amerika utara, jepang, PAL, lain-lain)
* Nilai kritik

Persiapan sumbu y

In [19]:
#numerisasi label (identifikasi jenis label)
jnslbl = lbl.copy()
jnslbl.info()

<class 'pandas.core.series.Series'>
RangeIndex: 64016 entries, 0 to 64015
Series name: Label
Non-Null Count  Dtype 
--------------  ----- 
64016 non-null  object
dtypes: object(1)
memory usage: 500.2+ KB


In [38]:
#Persiapan pembagian data (untuk kluster)
ycls = jnslbl #masih kata-kata
#pelabelan dengan 'ohen.onehotencoder' (pembentukan ulang)
bntkbaru, label = pd.factorize(ycls, sort=True, use_na_sentinel=False)
#inspeksi hasil
print(bntkbaru)
print(label)

[2 2 2 ... 1 1 1]
Index(['game dengan tujuan utama pencari duit',
       'game favortit dengan keuntungan rendah',
       'game kurang berkenan dengan fokus pencarian keuntungan'],
      dtype='object')


sampel kolom pelabelan

In [41]:
#pembentukan ulang
enkoding = bntkbaru.ravel()
#inisiasi ohen
encoder = lenco() #Untuk proses inti
#penyesuaian data
lblohen = encoder.fit_transform(enkoding)

In [44]:
#konversi ke df untuk kluster jadi
dflbl = pd.DataFrame(lblohen, columns=['Biner1'], copy=True)
#cek hasil
smpllbl = dflbl.head(7)
tiplbl = dflbl.info()
print(smpllbl) #inspeksi hasil

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Biner1  64016 non-null  int64
dtypes: int64(1)
memory usage: 500.2 KB
   Biner1
0       2
1       2
2       2
3       0
4       2
5       0
6       0


Nilai ini **(ycls)** menjadi komponen untuk penyatuan

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [45]:
#proses scaling untuk sumbu X (robust)
rbsscal = RobustScaler(unit_variance=True) #untuk robust
stdscal = StandardScaler() #untuk standar
mmscal = MinMaxScaler(clip=True) #untuk indikator 'minmax'
xclscl = rbsscal.fit_transform(xcls) #untuk tahap 1
xclstd = stdscal.fit_transform(xclscl) #untuk tahap 2
#pengubahan ke data frame untuk evaluasi
dfxclstd = pd.DataFrame(xclstd, columns=xcls.columns,copy=True)

In [46]:
#inspeksi kolom
print(dfxclstd.head(7))

   total_sales   na_sales  jp_sales  pal_sales  other_sales  critic_score
0    43.291200  25.764981  9.890744  52.679756    48.324018      1.518277
1    41.299748  24.499722  5.872512  51.928485    46.769901      1.711821
2    34.361784  34.091206  4.533102  29.283037    27.498858      1.647306
3    33.740793  36.744169  0.308807  28.424442    21.904039      1.647306
4    32.091956  24.989500  3.914912  32.288121    37.756026      0.679590
5    31.513792  36.784984  1.030028  22.843574    20.505334      1.066677
6    31.342484  39.601207  0.823965  19.838490    17.552513      1.131191


6 kolom di dataframe 'dfxclstd' menjadi aspek **"xclstd"**

In [47]:
#deklarasi ycl
ycl = dflbl.copy()

Nilai salinan dflbl menjadi **'ycl'**

In [77]:
#deklarasi nilai X dan y
X_train, X_test, y_train, y_test = train_test_split(xclstd, ycl, test_size=0.25, random_state=47, stratify=ycl) #untuk cek kondisi 1
X_train2, X_test2, y_train2, y_test2 = train_test_split(xclstd, ycl, test_size=0.225, random_state=47, stratify=ycl) #untuk KOndisi 2

In [84]:
#Berat sampel di latihan
brtsmplth = csw(class_weight='balanced', y=y_train) #dipakai di kondisi 1
brtsmplth2 = csw(class_weight='balanced', y=y_train2) #dipakai di kondisi 2

In [85]:
#Inisiasi 'berat sampel' di data tes
brtsampel = csw(class_weight='balanced', y=y_test) #dipakai di kondisi 1
brtsampel2 = csw(class_weight='balanced', y=y_test2) #dipakai di kondisi 2

- Tujuan pemasan berat di latihan adalah penstabilan data dengan indisitas
- Indisitas dengan '1' dan '2' menyatakan kondisi rasio

Rincian Percobaan
* Percobaan pertama untuk kedua metode berlangsung dengan rasio (3/1) (1)
* Percobaan kedua dengan rasio (31/9)

In [100]:
#Fungsi penentu akurasi melalui KNN
def train_and_evaluate(k):
    model = KNeighborsClassifier(n_neighbors=k, weights='distance', leaf_size=35,
                                 metric='seuclidean',p=1,metric_params={'V': np.var(X_train, axis=0)})
    #Tahap 'ravelisasi'
    y_trainrav = y_train.values.ravel()
    model.fit(X_train, y_trainrav)
    #Prediksi
    y_pred = model.predict(X_test)

    # Pembulatan metrik
    accuracy = accuracy_score(y_test, y_pred, sample_weight=brtsampel) # Akurasi yang dinormalkan
    precision = precision_score(y_test, y_pred, average='weighted', sample_weight=brtsampel, zero_division=0) # Penyesuaian di presisi
    recall = recall_score(y_test, y_pred, average='weighted',sample_weight=brtsampel, zero_division=0) #Penyesuaian di kondisi recall
    f1 = f1_score(y_test, y_pred, average='weighted',sample_weight=brtsampel, zero_division=0) #Penyesuaian di skor F1

    # Print metrics
    print(f"Total Tetangga k={k}")
    print(f"Nilai Akurasi: {accuracy:.4f}, Presisi: {precision:.4f}, Aspek Recall: {recall:.4f}, Skor F1: {f1:.4f}")

    # Classification report
    report = classification_report(y_test, y_pred, sample_weight=brtsampel, zero_division=0)

    if (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
       (accuracy >= 0.92 and precision >= 0.92 and recall >= 0.92 and f1 >= 0.92):
        print("Hasil sangat bagus untuk disimpan bisa disimpan.")
        return (k, accuracy, precision, recall, f1, y_pred, report)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
         (accuracy < 0.92 and precision < 0.92 and recall < 0.92 and f1 < 0.92):
        print("Hasil masih bisa disimpan.")
        return (k, accuracy, precision, recall, f1, y_pred, report)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall < 0.87 and f1 < 0.87) or \
         (accuracy > 0.92 and precision > 0.92 and recall < 0.92 and f1 < 0.92):
         print("Ada masalah di recall dan nilai f1.")
         return None
    else:
        print("Hasil tidak pantas disimpan.")
        return None
  #ke pembentukan matriks

In [101]:
#Fungsi evaluasi matriks
def generate_confusion_matrix(y_true, y_pred, brtsampel):
    cm = cmatr(y_true, y_pred, sample_weight=brtsampel, normalize='all') #Bisa matriks biasa atau multi matriks
    print("Confusion Matrix:")
    cm_rounded = np.round(cm.astype(float), decimals=4)  # Pembulatan untuk kemudahan nilai
    print("Hasil Confusion Matriks:")
    print(cm_rounded)
    return cm_rounded
#ke eksekusi asli

In [102]:
#Deklarasi parameter batas di KNN
param_grid = range(340,350)
#Paralelisasi melalui KNN
if __name__ == '__main__':
    with Pool() as pool:
        results = pool.map(train_and_evaluate, param_grid)

    # Filter out empty results
    filtered_results = [result for result in results if result is not None]

    # Get the best result
    best_result = max(filtered_results, key=lambda x: x[1], default=None)

    if best_result:
        # Pembulatans
        rounded_best_result = (
            best_result[0],  # Nilai awal
            round(best_result[1], 3),  # Pembulatan 1
            round(best_result[2], 3),  # Pembulatan 2
            round(best_result[3], 3),  # Pembulatan 3
            round(best_result[4], 3),  # Pembulatan 4
            best_result[5],  # Predictions (y_pred)
            best_result[6]   # Informasi tambahan
        )

        # Simpan nilai terbaik dari dill
        with open('best_knn_result.dill', 'wb') as f:
            dill.dump(rounded_best_result, f)

        _, _, _, _, _, y_pred, _ = rounded_best_result
        cm = generate_confusion_matrix(y_test, y_pred, brtsampel)

        # Simpan hasil matriks terbaik saja
        with open('best_confusion_matrix.dill', 'wb') as cm_file:
            dill.dump(cm, cm_file)

        # Print the best rounded result
        print("Best Result:", rounded_best_result)
#eksekusi KNN

Total Tetangga k=342
Nilai Akurasi: 0.4282, Presisi: 0.4374, Aspek Recall: 0.4282, Skor F1: 0.4101
Total Tetangga k=340
Nilai Akurasi: 0.4284, Presisi: 0.4377, Aspek Recall: 0.4284, Skor F1: 0.4104
Hasil masih bisa disimpan.
Hasil masih bisa disimpan.
Total Tetangga k=341
Nilai Akurasi: 0.4284, Presisi: 0.4377, Aspek Recall: 0.4284, Skor F1: 0.4104
Hasil masih bisa disimpan.
Total Tetangga k=343
Nilai Akurasi: 0.4285, Presisi: 0.4378, Aspek Recall: 0.4285, Skor F1: 0.4104
Hasil masih bisa disimpan.
Total Tetangga k=344
Nilai Akurasi: 0.4284, Presisi: 0.4377, Aspek Recall: 0.4284, Skor F1: 0.4104
Hasil masih bisa disimpan.
Total Tetangga k=346
Nilai Akurasi: 0.4282, Presisi: 0.4373, Aspek Recall: 0.4282, Skor F1: 0.4101
Hasil masih bisa disimpan.
Total Tetangga k=345
Nilai Akurasi: 0.4283, Presisi: 0.4375, Aspek Recall: 0.4283, Skor F1: 0.4103
Hasil masih bisa disimpan.
Total Tetangga k=347
Nilai Akurasi: 0.4282, Presisi: 0.4375, Aspek Recall: 0.4282, Skor F1: 0.4102
Hasil masih bisa di

In [92]:
#Tahap pertama RFC (pelacakan nilai ccp alpha)
nialpf = np.arange(1.75e-4, 0.175, 1.75e-4) #Nilai alfa
itmaks = 420
tolerance = 1.8e-4
best_alpha = None
best_accuracy = -np.inf
best_f1 = -np.inf
previous_best_accuracy = None
#ke deklarasi berat sampel

In [ ]:
#Inisiasi joblib
filename = '/tmp/dataset.joblib'
joblib.dump(np.asfortranarray(X_train), filename)
X_train_mmap = joblib.load(filename, mmap_mode='r')

In [ ]:
#Persiapan berat sampel
klsbrt = np.unique(y_train) #kelas berat di RFC
brtsmplth = csw(class_weight='balanced', y=y_train) #nilai deklarasi
dictbrt = dict(zip(klsbrt, brtsmplth)) #dideklarasikan sebagai kampus

In [ ]:
#Fungsi inspeksi nilai ccp alpha optimal
def evaluasi_alfa(alpha):
    rfc = RandomForestClassifier(n_estimators=121, max_depth=8, n_jobs=3,random_state=50, ccp_alpha=alpha, class_weight=dictbrt ,warm_start=True)
    #Perataan menjadi satu dimensi
    y_trainr = y_train.values.ravel()

    rfc.fit(X_train_mmap, y_trainr, sample_weight=brtsmplth)

    y_pred = rfc.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    # Pembulatan akurasi
    accuracy_rounded = round(accuracy, 3)
    # Penghindaran Peringatan di normalisasi
    f1 = f1_score(y_test, y_pred, average='weighted', sample_weight=brtsampel, zero_division=0)  # Nilai f1
    precision = precision_score(y_test, y_pred, average='weighted', sample_weight=brtsampel, zero_division=0)  # indikator presisi
    recall = recall_score(y_test, y_pred, average='weighted', sample_weight=brtsampel, zero_division=0)  # Nilai Recall

    return alpha, accuracy_rounded, f1, precision, recall  # Return values

# Iterasi melalui jangkauan nilai alfa
for iteration in range(itmaks):
    with Pool() as pool:
        results = pool.map(evaluasi_alfa, nialpf)

    for alpha, accuracy, f1, precision, recall in results:
        if accuracy > best_accuracy or (accuracy == best_accuracy and f1 > best_f1):
            best_alpha = alpha
            best_accuracy = accuracy
            best_f1 = f1

        # Pemeriksaan untuk nilai minimal (atau optimal)
        if (accuracy >= 0.92 and precision >= 0.92 and
            recall >= 0.92 and f1 >= 0.92) or \
           (accuracy >= 0.87 and precision >= 0.87 and
            recall >= 0.87 and f1 >= 0.87):
            print(f"Optimal metrics reached with ccp_alpha: {alpha}, "
                  f"Akurasi: {accuracy:.3f}, F1 Score: {f1:.2f}, "
                  f"Precision: {precision:.2f}, Recall: {recall:.2f}. Stopping search.")
            break

    # Pengaturan Nilai alfa
    nialpf = np.linspace(max(0.001, best_alpha - 0.005), min(0.1, best_alpha + 0.005), num=10)

    # Check for tolerance to stop early
    if iteration > 0 and abs(best_accuracy - previous_best_accuracy) < tolerance:
        print("Penghentian terjadi karena sudah melewati batas toleransi.")
        break

    previous_best_accuracy = best_accuracy

# Final results output
print(f'Nilai ccp_alpha terbaik: {best_alpha}, Accuracy: {best_accuracy:.3f}, F1 Score: {best_f1:.2f}')
#ke fungsi RFC

In [ ]:
#Tahap Perhitungan
final_model = RandomForestClassifier(random_state=50, ccp_alpha=best_alpha, class_weight=brtsmplth ,warm_start=True)
final_model.fit(X_train_mmap, y_train, sample_weight=brtsmplth)
y_pred_final = final_model.predict(X_test)
#Ke evaluasi

In [ ]:
#evaluasi melalui laporan dan 'confusion matrix'
conf_matrix = cmatr(y_test, y_pred_final, sample_weight=brtsampel, normalize='all')
class_report = classification_report(y_test, y_pred_final, sample_weight=brtsampel, zero_division=0)
#pencetakan nilai matriks
print("Confusion Matrix:")
print(conf_matrix)
print("\nLaporan Klasifikasi:")
print(class_report)

Confusion Matrix:
[[3811 1301  763]
 [2103 2396  787]
 [2008 1509 1326]]

Laporan Klasifikasi:
                                                        precision    recall  f1-score   support

                 game dengan tujuan utama pencari duit       0.48      0.65      0.55      5875
                game favortit dengan keuntungan rendah       0.46      0.45      0.46      5286
game kurang berkenan dengan fokus pencarian keuntungan       0.46      0.27      0.34      4843

                                              accuracy                           0.47     16004
                                             macro avg       0.47      0.46      0.45     16004
                                          weighted avg       0.47      0.47      0.46     16004



Tulis narasi atau penjelasan algoritma yang Anda gunakan.
### Rincian pemakaian algoritma KNN (K-nearest Neighboorhood)
1. Persiapan numerisasi label **'labelencoder'**
2. Proses standardisasi kolom numerik dengan **"Robustscaler","Standard Scaler", dan "Minmax Scaler"**
3. Eksekusi dengan teknik KNN melalui beberapa tahap :
#### 3.a. Proses Penentuan parameter di metode K-nearestNeighborhood dengan pemberat di data latihan
#### 3.b. Proses Paralelisasi untuk eksekusi dan evaluasi
#### 3.c. Tahap pencetakan Nilai akurasi, presisi, 'recall', dan skor f1
### Rincian pemkaian algoritma RFC (Random Forest Classifier)
* Persiapan numerisasi label dengan **'onehotencoding'**
* proses standardisasi dengan metode sama (agar lebih adil)
* Eksekusi fungsi yang terdiri dari
** a. Pelatihan RFC dengan RandomSearch CV untuk modifikasi nanti
** b. Tahapan paralelisasi untuk RFC
** c. Tahap pencetakan nilai RFC

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## Evaluasi algoritma antara Hasil 'tetangga terdekat' dengan teknik 'randomforest'
### Evaluasi dengan teknik KNN
#### 1. Hasil mengalmi fluktuasi rendah nilai akurasi presisi indeks 'recall' dan skor f1
#### 2. Nilai masih belum mencapai akurasi minimal walau nilai dipasang pemberat
#### 3.  
### Evaluasi dnegan teknik RFC
#### a. Nilai masih terlalu rendah baik tanpa pemberatan ataupun pemberatan
#### b.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

## Modifikasi di teknik KNN

In [ ]:
#Inisiasi grid search di KNN
def tuned_train_and_evaluate(X_train, y_train, X_test, y_test, brtsampel):
  # Definisikan nilai sampel
    knn = KNeighborsClassifier(weights='distance', leaf_size=35,
                                 metric='seuclidean',verbose=1,p=1,metric_params={'V': np.var(X_train, axis=0)})

    # Definisi pemberat dalam parameter
    brtsmplth = csw(class_weight='balanced', y=y_train)
    #eksekusi parameter
    param_dist = {
        'n_neighbors': range(250, 260),  # Penyetelan
        'weights': ['uniform', brtsmplth],
        'metric': ['seuclidean', 'cosine']
    }

    # Initialize RandomizedSearchCV
    random_search = RandomizedSearchCV(knn, param_distributions=param_dist, verbose=1,n_iter=20, cv=5, random_state=42)

    # Penyesuaian model dengan kondisi tertentu
    random_search.fit(X_train, y_train) #Modifikasi Joblib

    # Prediksi model terbaik
    y_pred = random_search.predict(X_test)

    # Pembulatan metrik
    accuracy = accuracy_score(y_test, y_pred, sample_weight=brtsampel)  # Akurasi yang dinormalkan
    precision = precision_score(y_test, y_pred, average='weighted', sample_weight=brtsampel)  # Penyesuaian di presisi
    recall = recall_score(y_test, y_pred, average='weighted', sample_weight=brtsampel)  # Penyesuaian di kondisi recall
    f1 = f1_score(y_test, y_pred, average='weighted', sample_weight=brtsampel)  # Penyesuaian di skor F1

    # Cetak Metrik
    print(f"Best Parameters: {random_search.best_params_}")
    print(f"Nilai Akurasi: {accuracy:.4f}, Presisi: {precision:.4f}, Aspek Recall: {recall:.4f}, Skor F1: {f1:.4f}")

    # Classification report
    report = classification_report(y_test, y_pred)

    if (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
       (accuracy >= 0.92 and precision >= 0.92 and recall >= 0.92 and f1 >= 0.92):
        return (random_search.best_params_['n_neighbors'], accuracy, precision, recall, f1, y_pred, report)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
         (accuracy < 0.92 and precision < 0.92 and recall < 0.92 and f1 < 0.92):
        print("Hasil masih bisa disimpan.")
        return (random_search.best_params_['n_neighbors'], accuracy, precision, recall, f1, y_pred, report)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall < 0.87 and f1 < 0.87) or \
         (accuracy > 0.92 and precision > 0.92 and recall < 0.92 and f1 < 0.92):
         print("Ada masalah di recall dan nilai f1.")
         return None
    else:
        print("Hasil tidak pantas disimpan.")
        return None
  #ke tuning lanjutan

In [ ]:
#Lanjutan tuning di KNN (matriks)
def tuned_confusion_matrix(y_true, y_pred):
    cm = cmatr(y_true, y_pred,sample_weight=brtsampel, normalize='all')
    print("Confusion Matrix:")
    cm_rounded = np.round(cm.astype(float), decimals=4)  # Pembulatan untuk kemudahan nilai
    print("Hasil Confusion Matriks:")
    print(cm_rounded)
    return cm_rounded
#ke eksekusi akhir

In [ ]:
#Eksekusi dengan Randomized search
if __name__ == '__main__':
    # Definisi pemberatan
    result = tuned_train_and_evaluate(X_train, y_train, X_test, y_test, brtsampel)

    if result:
        k_value = result[0]
        rounded_results = tuple(round(x, 3) if isinstance(x, float) else x for x in result)

        # Simpan nilai terbaik dari dill
        with open('tuned_knn_result.dill', 'wb') as f:
            dill.dump(rounded_results, f)

        _, _, _, _, _, y_pred, _ = rounded_results
        cm2 = generate_confusion_matrix(y_test, y_pred)

        # Simpan hasil matriks terbaik saja
        with open('tuned_confusion_matrix.dill', 'wb') as cm_file:
            dill.dump(cm2, cm_file)

        # Print the best rounded result
        print("Best Result:", rounded_results)

Best Parameters: {'weights': 'distance', 'n_neighbors': 198, 'metric': 'manhattan'}
Nilai Akurasi: 0.4387, Presisi: 0.4443, Aspek Recall: 0.4387, Skor F1: 0.4244
Hasil masih bisa disimpan.
Confusion Matrix:
Hasil Confusion Matriks:
[[3383.3137 1222.2062  729.1468]
 [2211.1719 2291.9084  831.5863]
 [2325.311  1663.2969 1346.0588]]
Best Result: (198, 0.439, 0.444, 0.439, 0.424, array(['game favortit dengan keuntungan rendah',
       'game kurang berkenan dengan fokus pencarian keuntungan',
       'game favortit dengan keuntungan rendah', ...,
       'game dengan tujuan utama pencari duit',
       'game favortit dengan keuntungan rendah',
       'game dengan tujuan utama pencari duit'], dtype=object), '                                                        precision    recall  f1-score   support\n\n                 game dengan tujuan utama pencari duit       0.46      0.63      0.54      5875\n                game favortit dengan keuntungan rendah       0.44      0.43      0.44      5286

Aspek modifikasi:
1. Implementasi dua metode pemberat (seuclidian dan nilai parameter 'brtsmplth')
2. Nilai cosine bertujuan untuk inspeksi sudut

## Modifikasi metode RFC

In [ ]:
#Tahap awal di RCT (deklarasi parameter)
rcfparam = {'ccp_alpha': np.arange(1.75e-4, 0.175, 1.75e-4),'estimator':[100, 150, 200],
            'max_depth':[1,10,20,30],'min_samples_split': [2, 5, 10],'class_weight': ['balanced', brtsmplth]} #Parameter

In [ ]:
#Lanjutan di RCT (Fungsi)
def rfc_randomized(X_train, y_train):
#Persiapan berat sampel
    klsbrt = np.unique(y_train) #kelas berat di RFC
    brtsmplth = csw(class_weight='balanced', y=y_train) #nilai deklarasi
    dictbrt = dict(zip(klsbrt, brtsmplth)) #dideklarasikan sebagai kampus
    #Inisiasi rfc dalam randomized
    rfc2 = RandomForestClassifier(random_state=50, class_weight=dictbrt ,warm_start=True, verbose=1)
    #Penyetelan Randomsearch CV
    random_search = RandomizedSearchCV(estimator=rfc2, param_distributions=rcfparam,
                                       n_iter=100,scoring='f1_weighted',
                                       cv=6, verbose=2,
                                       random_state=42,
                                       n_jobs=-1)
  #ke persiapan verbose

In [ ]:
#Pemangilan hasil
randmod , randparam = rfc_randomized(X_train_mmap, y_train)
#Prediksi dengan randomized search
ypred_rnd = randmod.predict(X_test)
#Evaluasi hasil randomizedsearch melalui matriks dan laporan
cmatrand = cmatr(y_test, y_pred_final, sample_weight=brtsampel, normalize='all') #pasca modifikasi
laporan = classification_report(y_test, y_pred_final, sample_weight=brtsampel, zero_division=0)

# Print results
print(f'Best Parameters: {randmod}')
print("Inspeksi Matriks Kebingungan:")
print(cmatrand)
print("\nLaporan Klasifikasi:")
print(laporan)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
#Inspeksi di metode K-NearestNeighborhood (Normal)
hslcmatk = cm #Hasil dari K-Nearest Neighborhood
hslcmatr = conf_matrix #Hasil dari Randomforest classifier

In [ ]:
#Modifikasi Randomizedsearch CV
hslmoda = cm2 #Hasil dari Randomizedsearch CV untuk K-nearest
hslmodb = cmatrand #Hasil dari Randomizedsearch CV untuk RFC
#ke ilustrasi matriks

In [ ]:
#Inspeksi metode 'KNN' melalui ilustrasi matriks (normal)
plt.figure(figsize=(10, 7))
sns.heatmap(hslcmatk, annot=True, fmt='.4f', cmap='Blues',
            xticklabels=['Kelas 1', 'Kelas 2', 'Kelas 3'],  # Adjust according to your classes
            yticklabels=['Kelas 1', 'Kelas 2', 'Kelas 3'])  # Adjust according to your classes
plt.ylabel('Label sesungguhnya')
plt.xlabel('Label terprediksi')
plt.title('Inspeksi Matrik melalui metode KNN')
plt.show()

In [ ]:
#Dengan modifikasi 'RandomizedsearchCV'


In [ ]:
#Inspeksi di metode RandomForestClasifier


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.